In [83]:
import math
from datetime import date, datetime, timedelta
from dateutil import relativedelta
from pprint import pprint
from random import randrange

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame
from typing import TypedDict

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

In [91]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


def get_previous_datetime_by(standard_time, year):
    weeks = year * 52  # 正確には52週ではないが妥協
    return standard_time - timedelta(weeks=weeks)


def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def ATR(arr: pd.DataFrame) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["atr"]


def SMA(arr: pd.DataFrame, payload=200) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["open_" + str(payload) + "_sma"]


def BB(arr: pd.DataFrame) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    sdf.BOLL_PERIOD = 200
    sdf.BOLL_STD_TIMES = 3
    return (sdf["boll"], sdf["boll_ub"], sdf["boll_lb"])


def DC(arr: pd.DataFrame, payload=20):
    count = 0

    max = []
    min = []
    middle = []

    df = convert_df_to_stock_df(arr).copy()
    for index, data in df.iterrows():

        if payload > count:
            # 計算できていない間は反応しないように適当に広げておく
            max.append(data["high"] + 10)
            min.append(data["low"] - 10)
            middle.append((data["high"] + data["low"]) / 2)
            count += 1
            continue

        # 当日は含めないので-1しとく
        range = df[count - payload : count - 1]
        middle.append((range["high"].max() + range["low"].min()) / 2)
        max.append(range["high"].max())
        min.append(range["low"].min())
        count += 1

    df["max"] = max
    df["min"] = min
    df["middle"] = middle

    # 以下はトレンド（何回最高を更新したか）の回数

    trend_arr = []
    trend = 0
    for index, data in df.iterrows():
        if data["max"] < data["high"]:
            if trend < 0:
                trend = 0
            else:
                trend += 1

        if data["min"] > data["low"]:
            if trend > 0:
                trend = 0
            else:
                trend -= 1
        trend_arr.append(trend)
    df["trend"] = trend_arr

    return (df["min"], df["max"])  # df["middle"], df["trend"]


class WalkFowardDates(TypedDict):
    in_sample_start: date
    in_sample_end: date
    out_sample_start: date
    out_sample_end: date


def create_walk_foward_test_date(base_date: date, in_sample_payload_year: int, out_sample_payload_year: int) -> WalkFowardDates:
    in_sample_start = base_date
    in_sample_end = in_sample_start + relativedelta.relativedelta(years=in_sample_payload_year, days=-1)

    out_sample_start = in_sample_end + relativedelta.relativedelta(days=1)
    out_sample_end = out_sample_start + relativedelta.relativedelta(years=out_sample_payload_year, days=-1)

    r: WalkFowardDates
    r = {
        'in_sample_start': in_sample_start,
        'in_sample_end': in_sample_end,
        'out_sample_start': out_sample_start,
        'out_sample_end': out_sample_end,
    }

    return r

In [3]:
def get_random_yf_data(tickers: str, start_str: str, end_str: str, payload_year: int):
    end = random_date(
        datetime.strptime(start_str, "%Y/%m/%d %H:%M:%S"),
        datetime.strptime(end_str, "%Y/%m/%d %H:%M:%S"),
    )

    start = get_previous_datetime_by(end, payload_year)

    # Valid start and end: YYYY-MM-DD
    # Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    # Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
    return yf.download(
        tickers=tickers,
        start=start,
        end=end,
        interval="1d",
        group_by="ticker",
    ).dropna()


def get_yf_data(tickers: str, start_str: str, end_str: str):
    return yf.download(
        tickers=tickers,
        start=start_str,
        end=end_str,
        interval="1d",
        group_by="ticker",
    ).dropna()

In [4]:
yf_datas = []
for name in range(1, 10, 1):
    yf_datas.append(
        get_random_yf_data("NDX", "2008/10/01 00:00:00", "2022/11/24 00:00:00", 3)
    )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [119]:
# My_Strategy(entry_payload_day=90,sell_payload_day=80,sma_payload=150)
class My_Strategy(Strategy):
    # 1week = 5day (取引所の営業日)
    entry_payload_day = 100
    sell_payload_day = 50
    sma_payload = 150

    atr_stop_loss = 250
    atr_take_profit = 250

    day = 0

    def init(self):
        self.atr = self.I(ATR, self.data.df)

        (self.dc_min, self.dc_max) = self.I(DC, self.data.df, self.entry_payload_day)

        (self.dc_half_min, self.dc_half_max) = self.I(
            DC, self.data.df, int(self.entry_payload_day / 2)
        )

        self.sma = self.I(SMA, self.data.df, self.sma_payload)

    def atr_unit(self):
        # 変数は1ATRが与える影響度: 0.01 ⇨ 1ATRで総資産の1%の変化
        unit = math.floor((self.equity * 10) / (self.atr[-1] * self.data.Close[-1]))
        return unit

    def next(self):
        self.day += 1

        # 計算できていない場合トレードしない
        if len(self.data.index) < self.sma_payload:
            return

        # 手仕舞い (半分のDCの高値または安値を下回った場合)
        if (self.dc_half_min[-1] > self.data.Low[-1] and self.position.is_long) or (
            self.data.High[-1] > self.dc_half_max[-1] and self.position.is_short
        ):
            self.position.close()
            return

        # 手仕舞い（SMA）
        if (self.sma[-1] > self.data.Close[-1] and self.position.is_long) or (
            self.sma[-1] < self.data.Close[-1] and self.position.is_short
        ):
            self.position.close()
            return

        # 買い注文
        if (
            self.sma[-1] < self.data.Close[-1]
            and self.data.Close[-1] > self.dc_max[-1]
            and not self.position.is_long
        ):
            self.buy(
                sl=self.data.Close[-1] - (self.atr[-1] * (self.atr_stop_loss / 100)),
                tp=self.data.Close[-1] + (self.atr[-1] * (self.atr_take_profit / 100)),
                # sl=self.data.Close[-1] * (1 - (self.stop_loss / 100)),
                # tp=self.data.Close[-1] * (1 + (self.take_profit / 100)),
                    )
            return

        # 売り注文
        if (
            self.sma[-1] > self.data.Close[-1]
            and self.dc_min[-1] > self.data.Close[-1]
            and not self.position.is_short
        ):
            # self.sell(
            #     sl=self.data.Close[-1] * (1 + (self.stop_loss / 100)),
            #     # sl=self.data.Close[-1] + (self.atr[-1] * (self.atr_stop_loss / 100)),
            #     tp=self.data.Close[-1] * (1 - (self.take_profit / 100)),
            # )
            return

In [6]:
result = []
for yf_data in yf_datas:
    bt = Backtest(
        yf_data,
        My_Strategy,
        cash=1000000000,
        commission=0.003,
        exclusive_orders=True,
    )
    result.append(bt.run())

print(pd.DataFrame(result)["SQN"])
print(pd.DataFrame(result)["SQN"].mean())

0     1.70
1    -0.32
2    -0.12
3    -0.51
4    -0.12
5     0.48
6    -0.05
7     1.39
8     0.58
Name: SQN, dtype: float64
0.3366111421265309


In [9]:
print(pd.DataFrame(result).iloc[8])

Start                                                   2018-09-06 00:00:00
End                                                     2020-09-02 00:00:00
Duration                                                  727 days 00:00:00
Exposure Time [%]                                                     75.90
Equity Final [$]                                              1430893639.93
Equity Peak [$]                                               1430893639.93
Return [%]                                                            43.09
Buy & Hold Return [%]                                                 66.65
Return (Ann.) [%]                                                     19.71
Volatility (Ann.) [%]                                                 28.13
Sharpe Ratio                                                           0.70
Sortino Ratio                                                          1.27
Calmar Ratio                                                           1.29
Max. Drawdow

In [114]:
# BTC-USD BEST SQN 3.03 My_Strategy(entry_payload_day=140,sell_payload_day=20,atr_take_profit=500,atr_stop_loss=600)
# SPY     BEST SQN 2.67 My_Strategy(entry_payload_day=100,sell_payload_day=80,atr_take_profit=500,atr_stop_loss=500)


bt = Backtest(
    get_yf_data("SPY", "2000-11-06", "2022-12-02"),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)

# 最適化
# optimize = bt.optimize(
#     maximize="SQN",
#     # stop_loss=range(2, 8, 1),
#     # take_profit=range(2, 10, 2),
#     # entry_payload_day=range(20, 200, 40),
#     # sell_payload_day=range(10, 100, 10),
#     # atr_take_profit=range(100, 800, 100),
#     # atr_stop_loss=range(100, 800, 100),
#     # constraint=lambda p: p.sell_payload_day < p.entry_payload_day,
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)

# 出力
# output = bt.run()
# print(output)
# bt.plot()



[*********************100%***********************]  1 of 1 completed


In [106]:
df = get_yf_data("SPY", "2000-11-06", "2022-12-02")

[*********************100%***********************]  1 of 1 completed


In [125]:
create_walk_foward_test_date(date(2013, 1, 1), 4, 2)

{'in_sample_start': datetime.date(2013, 1, 1),
 'in_sample_end': datetime.date(2016, 12, 31),
 'out_sample_start': datetime.date(2017, 1, 1),
 'out_sample_end': datetime.date(2018, 12, 31)}

In [128]:
payload = create_walk_foward_test_date(date(2005, 1, 1), 3, 2)


for payload in [
    create_walk_foward_test_date(date(2001, 1, 1), 5, 2),
    create_walk_foward_test_date(date(2008, 1, 1), 5, 2),
    create_walk_foward_test_date(date(2015, 1, 1), 5, 2),
]:
    bt = Backtest(
        df.query('"{}" <= index <= "{}"'.format(payload['in_sample_start'].strftime("%Y-%m-%-d"), payload['in_sample_end'].strftime("%Y-%m-%-d"))),
        My_Strategy,
        cash=10000000,
        commission=0.003,
        exclusive_orders=True,
    )

    optimize = bt.optimize(
        maximize="SQN",
        entry_payload_day=range(20, 200, 10),
        sma_payload=range(10, 100, 10),
        # atr_take_profit=range(100, 800, 100),
        # atr_stop_loss=range(100, 800, 100),
    )

    bt = Backtest(
        df.query('"{}" <= index <= "{}"'.format(payload['out_sample_start'].strftime("%Y-%m-%-d"), payload['out_sample_end'].strftime("%Y-%m-%-d"))),
        My_Strategy,
        cash=10000000,
        commission=0.003,
        exclusive_orders=True,
    )

    print(optimize['SQN'])
    print(optimize._strategy._params)
    output = bt.run(**optimize._strategy._params)
    print(output)
    bt.plot()

{'entry_payload_day': 80, 'sma_payload': 30}
{'entry_payload_day': 80, 'sma_payload': 30}
Start                     2006-01-03 00:00:00
End                       2007-12-31 00:00:00
Duration                    727 days 00:00:00
Exposure Time [%]                       29.08
Equity Final [$]                   9421928.64
Equity Peak [$]                   10410299.18
Return [%]                              -5.78
Buy & Hold Return [%]                   15.40
Return (Ann.) [%]                       -2.94
Volatility (Ann.) [%]                    4.71
Sharpe Ratio                             0.00
Sortino Ratio                            0.00
Calmar Ratio                             0.00
Max. Drawdown [%]                       -9.49
Avg. Drawdown [%]                       -2.22
Max. Drawdown Duration      382 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                                   13
Win Rate [%]                            46.15
Best Trade [%]                      

{'entry_payload_day': 40, 'sma_payload': 90}
{'entry_payload_day': 40, 'sma_payload': 90}
Start                     2013-01-02 00:00:00
End                       2014-12-31 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                       44.84
Equity Final [$]                   9297607.53
Equity Peak [$]                   10198898.37
Return [%]                              -7.02
Buy & Hold Return [%]                   40.72
Return (Ann.) [%]                       -3.58
Volatility (Ann.) [%]                    5.58
Sharpe Ratio                             0.00
Sortino Ratio                            0.00
Calmar Ratio                             0.00
Max. Drawdown [%]                       -9.16
Avg. Drawdown [%]                       -3.24
Max. Drawdown Duration      593 days 00:00:00
Avg. Drawdown Duration      200 days 00:00:00
# Trades                                   14
Win Rate [%]                            42.86
Best Trade [%]                      

{'entry_payload_day': 100, 'sma_payload': 30}
{'entry_payload_day': 100, 'sma_payload': 30}
Start                     2020-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                       41.19
Equity Final [$]                  10137669.92
Equity Peak [$]                   11066357.50
Return [%]                               1.38
Buy & Hold Return [%]                   46.20
Return (Ann.) [%]                        0.68
Volatility (Ann.) [%]                    7.08
Sharpe Ratio                             0.10
Sortino Ratio                            0.13
Calmar Ratio                             0.07
Max. Drawdown [%]                       -9.21
Avg. Drawdown [%]                       -1.95
Max. Drawdown Duration      340 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                   18
Win Rate [%]                            44.44
Best Trade [%]                    